# Домашнее задание 3
## Сравнение интересов аудитории телеканалов НТВ и Дождь с помощью тематического моделирования LDA

Задача:
Сравнить интересы аудитории телеканолов НТВ и Дождь с помощью методов тематического моделирования
1. Получить данные по аудитории из социальной сети ВК
2. Зарегистрировать приложение, получить app_id, access_token
3. Скачать данные по пользователям в каждой из групп (id групп ВК даны ниже, tvrain_id, ntv_id)
4. Взять небольшую выборку из каждой совокупности телезрителей(около 1000-2000 человек, т.к. 300k-400k слишком много), с которыми работать дальше
5. Обучить LDA модель на их подписках
6. По группам, на которые подписаны эти люди, полуичть ключевые слова групп, на которые они подписаны
7. Получить распределение интересов людей для каждой совокупности, сравнить на графике

In [1]:
import seaborn as sb
import sys  
import pandas as pd
import requests
import numpy as np
import datetime
import time
import pickle
import matplotlib.pyplot as plt
import time
%matplotlib inline

from bs4 import BeautifulSoup as bs
import urllib

In [2]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#### Для использования VK API необходимо создать приложение в VK

1. Создать приложение по адресу https://vk.com/apps?act=manage (кнопка "создать приложение")
2. При создании указать название, описание (можно любые), категория  - прочее. Тип - standalone приложение
3. В настройках получить **app_id**. App_id потребуется для получения access token
4. Авторизовать пользователя (получить access token) можно по адресу: https://vk.com/dev/first_guide, в правилах нас интересует пункт 3 **Авторизация пользователя**
5. После того, как ознакомитесь с авторизацией пользователя, скопируйте в адресную строку такой запрос https://oauth.vk.com/authorize?client_id=5490057&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.52, где число **5490057** замените на число, которое получите для вашего **app_id**
6. Нажмите Enter. Откроется окно с запросом прав. В нем отображаются название приложения, иконки прав доступа, и Ваши имя с фамилией. Нажмите «Разрешить». Вы попадете на новую страницу с предупреждением о том, что токен нельзя копировать и передавать третьим лицам. В адресной строке будет URL https://oauth.vk.com/blank.html, а после # Вы увидите дополнительные параметры — access_token, expires_in и user_id. Токен может выглядеть, например, так: 51eff86578a3bbbcb5c7043a122a69fd04dca057ac821dd7afd7c2d8e35b60172d45a26599c08034cc40a
7. Токен — это Ваш ключ доступа. При выполнении определенных условий человек, получивший Ваш токен, может нанести существенный ущерб Вашим данным и данным других людей. Поэтому очень важно не передавать свой токен третьим лицам
8. Поле expires_in содержит время жизни токена в секундах. 86400 секунд — это ровно сутки. Через сутки полученный токен перестанет действовать, для продолжения работы нужно будет получить новый по такому же алгоритму

In [40]:
# use your own app id and respective tokens

# скопируйте сюда ваши app_id и access_token, полученные по методу, описанному выше
app_id = 6753500
access_token = '91ae15b191ae15b191ae15b12e91c9196d991ae91ae15b1ca4085334a332913843de536'

# id групп ВК Дождя и НТВ
tvrain_id = 17568841
ntv_id = 28658784

In [41]:
# проверка работы API и авторизации пользователя. Если возникает ошибка, то, возможно, access token необходимо обновить
check_id = 1

# api call and test
def vk_get_response(method, parameters, token):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + token
#     print url
    return(requests.get(url).json())

answer = vk_get_response(
    'users.get', 'user_ids={0}&v=4.9&lang=ru'.format(check_id), access_token
)['response']
print(answer[0]['first_name'], answer[0]['last_name'])

Павел Дуров


### Получение подписчиков телеканалов НТВ и Дождь в VK

Получим объекты, которые содержат всю информацию о подпиичиках соответствующих групп (указанных в domains) и сохраним их на диск. Получим в итоге два файла - **ntv_subs** и **tvrain_subs** в формате **.pkl** - питоновский формат хранения данных

In [5]:
domains = ['ntv', 'tvrain']

for group_domain in domains:
    offset = 0
    group_id = group_domain
    fields = """sex,bdate,city,country,home_town,lists,domain,has_mobile,
    contacts,connections,education,universities,followers_count,occupation,last_seen,relation"""
    first_sample = vk_get_response(
        'groups.getMembers', 'group_id={0}&offset={1}&fields={2}&v=4.9&lang=ru'.format(
            group_id, offset, fields
        ), token=access_token
    )
    community_count = first_sample['response']['count']
    community_members = []
    for i in range(community_count // 1000 + 1):
        offset = i * 1000
        try:
            answer = vk_get_response(
                'groups.getMembers', 'group_id={0}&offset={1}&fields={2}&v=4.9&lang=ru'.format(
                    group_id, offset, fields), token=access_token
            )
            print("Offset: ", offset)
        except:
            print("Offset: ", offset, " Error")
        community_members += answer['response']['users']
    save_obj(community_members, '{}_subs'.format(group_domain))

Offset:  0
Offset:  1000
Offset:  2000
Offset:  3000
Offset:  4000
Offset:  5000
Offset:  6000
Offset:  7000
Offset:  8000
Offset:  9000
Offset:  10000
Offset:  11000
Offset:  12000
Offset:  13000
Offset:  14000
Offset:  15000
Offset:  16000
Offset:  17000
Offset:  18000
Offset:  19000
Offset:  20000
Offset:  21000
Offset:  22000
Offset:  23000
Offset:  24000
Offset:  25000
Offset:  26000
Offset:  27000
Offset:  28000
Offset:  29000
Offset:  30000
Offset:  31000
Offset:  32000
Offset:  33000
Offset:  34000
Offset:  35000
Offset:  36000
Offset:  37000
Offset:  38000
Offset:  39000
Offset:  40000
Offset:  41000
Offset:  42000
Offset:  43000
Offset:  44000
Offset:  45000
Offset:  46000
Offset:  47000
Offset:  48000
Offset:  49000
Offset:  50000
Offset:  51000
Offset:  52000
Offset:  53000
Offset:  54000
Offset:  55000
Offset:  56000
Offset:  57000
Offset:  58000
Offset:  59000
Offset:  60000
Offset:  61000
Offset:  62000
Offset:  63000
Offset:  64000
Offset:  65000
Offset:  66000
Offset: 

Offset:  166000
Offset:  167000
Offset:  168000
Offset:  169000
Offset:  170000
Offset:  171000
Offset:  172000
Offset:  173000
Offset:  174000
Offset:  175000
Offset:  176000
Offset:  177000
Offset:  178000
Offset:  179000
Offset:  180000
Offset:  181000
Offset:  182000
Offset:  183000
Offset:  184000
Offset:  185000
Offset:  186000
Offset:  187000
Offset:  188000
Offset:  189000
Offset:  190000
Offset:  191000
Offset:  192000
Offset:  193000
Offset:  194000
Offset:  195000
Offset:  196000
Offset:  197000
Offset:  198000
Offset:  199000
Offset:  200000
Offset:  201000
Offset:  202000
Offset:  203000
Offset:  204000
Offset:  205000
Offset:  206000
Offset:  207000
Offset:  208000
Offset:  209000
Offset:  210000
Offset:  211000
Offset:  212000
Offset:  213000
Offset:  214000
Offset:  215000
Offset:  216000
Offset:  217000
Offset:  218000
Offset:  219000
Offset:  220000
Offset:  221000
Offset:  222000
Offset:  223000
Offset:  224000
Offset:  225000
Offset:  226000
Offset:  227000
Offset: 

In [5]:
community_ntv = load_obj('ntv_subs')
community_tvrain = load_obj('tvrain_subs')

In [6]:
community_ntv_df = pd.DataFrame(community_ntv)
community_tvrain_df = pd.DataFrame(community_tvrain)

Сначала получим всех уникальных подписчиков НТВ и Дождя с помощью unique. Далее с помощью numpy.random необходимо выбрать небольшой sample (например, по 1000 из каждой группы) таких людей и объединить их вместе

In [7]:
ntv_uids = community_ntv_df.uid.unique().tolist()
tvrain_uids = community_tvrain_df.uid.unique().tolist()

In [28]:
ntv = np.random.choice(ntv_uids,1000)
rain = np.random.choice(tvrain_uids,1000)

In [10]:
# получить общий список людей из двух выборок НТВ и Дождя, всего должно быть в итоге около 2000 человек
#uids = np.append(ntv,rain)
#uids.size

2000

In [7]:
# получить подписки этих пользователей

def get_final_data(uids):
    print_counter = 0
    final_data = []

    for uid in uids[0:1000]:
        try:
            user_subs = vk_get_response(
                'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
            )
            time.sleep(0.3)
            final_data.append(user_subs)
        except:
            print("Error")
        if print_counter % 100 == 0:
            print("{0} profiles done".format(print_counter))
        print_counter += 1
        
    return final_data

In [8]:
def get_user_groups(final_data, uids):

    subs_list = []
    groups_freq_dict = {}
    top_n = 5

    for record, uid in zip(final_data, uids):
        try:
            user_subs = record
            if not user_subs.get('response'):
                user_subs = vk_get_response(
                    'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
                )
            subs_pd = pd.DataFrame(
                [
                    {
                        'groups_count': user_subs['response']['groups'].get('count'),
                        'groups_list': user_subs['response']['groups'].get('items'),
                        'follows_count':user_subs['response']['users'].get('count'),
                        'follows_list': user_subs['response']['users'].get('items'),
                    }
                ]
            )

            for group_id in user_subs['response']['groups'].get('items')[:top_n]:
                if groups_freq_dict.get(group_id):
                    groups_freq_dict[group_id] += 1
                else:
                    groups_freq_dict[group_id] = 1

            subs_pd['subs_count'] = subs_pd['groups_count'] + subs_pd['follows_count']
            subs_list.append(subs_pd)
        except:
            print(user_subs)
                
        if len(subs_list) % 100 == 0:
            print("Processed {0} users".format(len(subs_list)))
            
    return [groups_freq_dict, subs_list]

Самые популярные группы

In [16]:
sorted([(key, val) for key, val in groups_freq_dict.items()], key=lambda x: x[1], reverse=True)[:5]

[(17568841, 336),
 (28658784, 180),
 (15755094, 74),
 (29534144, 46),
 (40316705, 38)]

Загрузка постов со стен групп

In [44]:
def get_group_doc_dict(groups_freq_dict):

    group_doc_dict = {}
    counter = 0
    groups_freq_dict_top5 = groups_freq_dict
    check = False

    for group_id, freq in groups_freq_dict_top5.items():
        counter += 1
        try:
            check = vk_get_response(
                'wall.get',
                'owner_id={0}&count=100&fields=post_type,marked_as_ads&&v=4.9&lang=ru'.format(int(group_id) * -1),
                access_token
            )
            check = check['response']
            group_doc = ''
            if check[0] != 0:
                for post in check[1:]:
                    if post.get('marked_as_ads') != 1:
                        group_doc += post['text']
            group_doc_dict[group_id] = group_doc
        except:
            print("Response error. Group id {0}".format(group_id))
            print(check)
        if counter % 100 == 0:
            print("{0} groups extracted".format(counter))
        time.sleep(0.3)
  
    return group_doc_dict

In [11]:
from bs4 import BeautifulSoup
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
from pymorphy2 import MorphAnalyzer

[nltk_data] Downloading package stopwords to /Users/qb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/qb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
chrs_to_delete = string.punctuation + u'»' + u'«' + u'—' + u'“' + u'„' + u'•' + u'#'
translation_table = {ord(c): None for c in chrs_to_delete if c != u'-'}
units = MorphAnalyzer.DEFAULT_UNITS
morph = MorphAnalyzer(result_type=None, units=units)
PortSt = PorterStemmer()
stopw = set(
    [w for w in stopwords.words(['russian', 'english'])]
    + [u'это', u'году', u'года', u'также', u'етот',
       u'которые', u'который', u'которая', u'поэтому',
       u'весь', u'свой', u'мочь', u'eтот', u'например',
       u'какой-то', u'кто-то', u'самый', u'очень', u'несколько',
       u'источник', u'стать', u'время', u'пока', u'однако',
       u'около', u'немного', u'кроме', u'гораздо', u'каждый',
       u'первый', u'вполне', u'из-за', u'из-под',
       u'второй', u'нужно', u'нужный', u'просто', u'большой',
       u'хороший', u'хотеть', u'начать', u'должный', u'новый', u'день',
       u'метр', u'получить', u'далее', u'именно', u'апрель',
       u'сообщать', u'разный', u'говорить', u'делать',
       u'появиться', u'2016',
       u'2015', u'получить', u'иметь', u'составить', u'дать', u'читать',
       u'ничто', u'достаточно', u'использовать',
       u'принять', u'практически',
       u'находиться', u'месяц', u'достаточно', u'что-то', u'часто',
       u'хотеть', u'начаться', u'делать', u'событие', u'составлять',
       u'остаться', u'заявить', u'сделать', u'дело',
       u'примерно', u'попасть', u'хотя', u'лишь', u'первое',
       u'больший', u'решить', u'число', u'идти', u'давать', u'вопрос',
       u'сегодня', u'часть', u'высокий', u'главный', u'случай', u'место',
       u'конец', u'работать', u'работа', u'слово', u'важный', u'сказать']
)

In [13]:
url_start = 'http[s]?://'
url_end = (
    '(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
)
pattern = url_start + url_end

Обработка слов постов групп - трансформация в "хороший" вид. Нормализация и стэмминг, удаление стоп-слов

In [14]:
def get_clean_dict(group_doc_dict):

    group_clean_doc_dict = {}
    counter = 0

    for group_id, doc in group_doc_dict.items():
        soup = BeautifulSoup(doc, 'html.parser')
        body = ' '.join(
            [tag.string.replace('\\n', ' ').replace('\\r', ' ')
             for tag in soup.descendants if tag.string]
        )
        body = re.sub('\[.*?\]','', body)
        body = re.sub(pattern,'', body)
        if body != '':
            body_clean = body.translate(translation_table).lower().strip()
            words = word_tokenize(body_clean)
            tokens = []
            # stemming and text normalization
            for word in words:
                if re.match('^[a-z0-9-]+$', word) is not None:
                    tokens.append(PortSt.stem(word))
                elif word.count('-') > 1:
                    tokens.append(word)
                else:
                    normal_forms = morph.normal_forms(word)
                    tokens.append(normal_forms[0] if normal_forms else word)
            # remove stopwords and leave unique words only
            tokens = filter(
                lambda token: token not in stopw, sorted(set(tokens))
            )

            # remove all words with more than 3 chars
            tokens = filter(lambda token: len(token) > 3, tokens)
        else:
            tokens = []
        counter += 1
        if counter % 100 == 0:
            print("{0} docs processed".format(counter))
        group_clean_doc_dict[group_id] = tokens

    group_clean_doc_dict = {key: list(val) for key, val in group_clean_doc_dict.items()}
 
    return group_clean_doc_dict

### Обучение LDA модели

In [17]:
from gensim.corpora import TextCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

class ListTextCorpus(TextCorpus):

    def get_texts(self):
        for doc in self.input:
            yield doc

In [18]:
def get_lda(group_clean_doc_dict):
    mycorp = ListTextCorpus(input=group_clean_doc_dict.values())
    justlda = LdaModel(
    corpus=mycorp, num_topics=20, passes=12)

    print('LdaModel performance')
    for i in range(20):
        terms = justlda.get_topic_terms(i)
        print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))
        
    return justlda

In [19]:
def show_groups(justlda, group_clean_doc_dict):
    dummy_dict = {key: 0 for key in range(20)}
    mycorp = ListTextCorpus(input=group_clean_doc_dict.values())

    group_topics_dict_20 = {
        group_id: dict(list(dummy_dict.items()) + justlda.get_document_topics(mycorp.dictionary.doc2bow(text)))
        for group_id, text in group_clean_doc_dict.items()
    }
    check_pd_20 = pd.DataFrame.from_dict(group_topics_dict_20, orient='index')
    check_pd_20.head(10)
    #print("Group distribution by the most relevant topic")
    #print(pd.Series.round(check_pd_20.idxmax(axis=1).value_counts() * 1. / len(check_pd_20), 2))
    
    return check_pd_20

In [20]:
def show_most_response_group(justlda, group_clean_doc_dict, check_pd_20):
    mycorp = ListTextCorpus(input=group_clean_doc_dict.values())
    print("The most typical groups for every topic")
    for i in range(20):
        terms = justlda.get_topic_terms(i)
        print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))
        typical_groups = check_pd_20[i].sort_values(ascending=False).index[:10]
        for g in typical_groups:
            group_info = vk_get_response(
                'groups.getById', 'group_ids={0}&v=4.9&lang=ru'.format(g), access_token
            )
            print(group_info['response'][0]['name'] + ' ' + 'http://vk.com/club' + str(g))
            time.sleep(0.3)
        print()
        
    return typical_groups

In [29]:
ntv_final_data = get_final_data(ntv)

0 profiles done
100 profiles done
200 profiles done
300 profiles done
400 profiles done
500 profiles done
600 profiles done
700 profiles done
800 profiles done
900 profiles done


In [30]:
temp_list = get_user_groups(ntv_final_data, ntv)
ntv_groups_freq_dict = temp_list[0]
ntv_subs_list = temp_list[1]

{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '407182574'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '433890227'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '408476162'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oau

{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '430187909'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '474602058'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '139081114'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oau

{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '3000653'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '415739363'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '426505263'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Processed 600 users
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_para

{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '18912903'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '290102631'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'users.getSubscriptions'}, {'key': 'user_id', 'value': '144420598'}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
{'error': {'error_code': 15, 'error_msg': 'Access denied: this profile is private', 'request_params': [{'key': 'oaut

In [31]:
ntv_group_doc_dict = get_group_doc_dict(ntv_groups_freq_dict)

100 groups extracted
200 groups extracted
300 groups extracted
400 groups extracted
Response error. Group id 85039950
[93525, {'id': 367268, 'from_id': -40498005, 'to_id': -40498005, 'date': 1542259867, 'marked_as_ads': 0, 'post_type': 'post', 'text': '', 'is_pinned': 1, 'media': {'type': 'photo', 'owner_id': -40498005, 'item_id': 456313116, 'thumb_src': 'https://sun1-11.userapi.com/c543106/v543106720/3f34c/3TI49IomXco.jpg'}, 'attachments': [{'type': 'photo', 'photo': {'pid': 456313116, 'aid': -7, 'owner_id': -40498005, 'user_id': 100, 'src': 'https://sun1-11.userapi.com/c543106/v543106720/3f34c/3TI49IomXco.jpg', 'src_big': 'https://sun1-12.userapi.com/c543106/v543106720/3f34d/bvhFjqz5-6k.jpg', 'src_small': 'https://sun1-8.userapi.com/c543106/v543106720/3f34b/zF3Q3x_M0xo.jpg', 'src_xbig': 'https://sun1-13.userapi.com/c543106/v543106720/3f34e/EHb92Rzwva0.jpg', 'src_xxbig': 'https://sun1-5.userapi.com/c543106/v543106720/3f34f/Pp5g6C8VqCM.jpg', 'width': 564, 'height': 844, 'text': '', 'cr

500 groups extracted
600 groups extracted
700 groups extracted
800 groups extracted
900 groups extracted
1000 groups extracted
1100 groups extracted
1200 groups extracted
1300 groups extracted
1400 groups extracted
1500 groups extracted
1600 groups extracted
1700 groups extracted
1800 groups extracted
1900 groups extracted
2000 groups extracted
2100 groups extracted
2200 groups extracted
2300 groups extracted
2400 groups extracted
2500 groups extracted
2600 groups extracted
2700 groups extracted
2800 groups extracted
2900 groups extracted


In [32]:
ntv_group_clean_doc_dict = get_clean_dict(ntv_group_doc_dict)

100 docs processed
200 docs processed
300 docs processed
400 docs processed
500 docs processed
600 docs processed
700 docs processed
800 docs processed
900 docs processed
1000 docs processed
1100 docs processed
1200 docs processed
1300 docs processed
1400 docs processed
1500 docs processed
1600 docs processed
1700 docs processed
1800 docs processed
1900 docs processed
2000 docs processed
2100 docs processed
2200 docs processed
2300 docs processed
2400 docs processed
2500 docs processed
2600 docs processed
2700 docs processed
2800 docs processed
2900 docs processed


In [34]:
ntv_justlda = get_lda(ntv_group_clean_doc_dict)

LdaModel performance
0 раунд досрочно весовой нурмагомедов макгрегор энтони конором следующий турнир нурмагомед
1 кэшбэкома амвросий архимандрит святитель диавол присно одигитрия евангелия паисий обстояние
2 побеждать ✅заходить пара покер выигрывать ✅играть pokerdom парень начинать финалист
3 лайк 6700 подарок тeбить любить rolex девушка телефон eсть временно
4 раста обваливать спагетти -лука -яйцо аппетитасалат паприка бекон панировочный тушиться
5 вода масло простой способ приготовление тонкий вкусный вариант рецепт полезный
6 чтoб ecть кoгдa тoлькo тeбить live мeнить тeбe night дaжe
7 light звучание интeресный matt alex buuren hold smith kearney matiss
8 человек 2018 жизнь крутой фэнтези серия любить фантастика приключение жанр
9 слева коммент видно 3040 фото вязаный спица наборный плохо основное
10 мужчина женщина лайк добрый фото боль девушка писать понравиться завтра
11 переодеться помощь результат сумма бабка помочь город момент катюша константин
12 оставить добрый роза ждать до

In [121]:
ntv_check_pd_20 = show_groups(ntv_justlda, ntv_group_clean_doc_dict)

print("Group distribution by the most relevant topic")
ntv_group_index_percent = pd.Series.round(ntv_check_pd_20.idxmax(axis=1).value_counts() * 1. / len(ntv_check_pd_20), 2)
print(ntv_group_index_percent)

Group distribution by the most relevant topic
14    0.49
15    0.13
13    0.09
5     0.07
0     0.04
18    0.04
6     0.02
8     0.01
11    0.01
16    0.01
4     0.01
10    0.01
7     0.01
3     0.01
12    0.01
2     0.01
1     0.01
17    0.01
9     0.01
19    0.01
dtype: float64


In [47]:
ntv_typical_groups = show_most_response_group(ntv_justlda, ntv_group_clean_doc_dict, ntv_check_pd_20)

The most typical groups for every topic
0 раунд досрочно весовой нурмагомедов макгрегор энтони конором следующий турнир нурмагомед
Зуб пацана http://vk.com/club57055326
Элитный Юмор http://vk.com/club108477741
мой личный дневние http://vk.com/club83177910
Уютный декор для дома. Handmade и рукоделие http://vk.com/club28587219
Пригон авто из Европы в ДНР ЛНР | Import Cars http://vk.com/club126209212
Fairwind http://vk.com/club136119979
Замедленная съемка | Gif http://vk.com/club133814709
O h , y e s http://vk.com/club90937359
Хочу всё знать http://vk.com/club91317441
Овсянка, сэр! http://vk.com/club66678575

1 кэшбэкома амвросий архимандрит святитель диавол присно одигитрия евангелия паисий обстояние
ЕБАТЬ MOZGI http://vk.com/club155770158
📌{ƏЙГЕРІМ ҚАЛАУБАЕВА}📌 http://vk.com/club149043700
Маникюр | Дизайн ногтей 2018 http://vk.com/club152114786
Как подкачаться? http://vk.com/club38127593
Мужские мысли http://vk.com/club25397178
Мужское Братство http://vk.com/club44155000
Мужской клуб ht

Прямая линия с Владимиром Путиным http://vk.com/club115987924
Правда | Гомель http://vk.com/club54192348
БНК http://vk.com/club26478611
КАНДАЛАКША ОНЛАЙН http://vk.com/club68142091
ТВ - Сокол http://vk.com/club58487285
Самара http://vk.com/club1528722
Телеканал «ЗВЕЗДА» http://vk.com/club6384628
Вечерний Бишкек http://vk.com/club105714344
Новости RT на русском http://vk.com/club40316705
Типичный Донецк http://vk.com/club23502694

16 appl edit full видео годовой подписка 2018 video iphon цифровой
FACE http://vk.com/club95470601
#FG Фурри для парней http://vk.com/club101666543
Своя Культура http://vk.com/club39674086
Инстаграм спорстменов http://vk.com/club138090979
Тото Кутуньо | Toto Cutugno http://vk.com/club144259022
MANDARIN GIF http://vk.com/club151201230
Bananastreet — клубная музыка http://vk.com/club183079
UFC | Жизнь бойца http://vk.com/club129386072
Адский Шутник http://vk.com/club95128257
Прогнозы от Best Capper in Live http://vk.com/club49884862

17 соңғ алдын басқ үлкен кел

In [60]:
save_obj(ntv_group_doc_dict, 'ntv_group_doc_dict')
save_obj(ntv_groups_freq_dict, 'ntv_groups_freq_dict')
save_obj(ntv_final_data, 'ntv_final_data')
save_obj(ntv_subs_list, 'ntv_subs_list')
save_obj(ntv_group_clean_doc_dict, 'ntv_group_clean_doc_dict')
save_obj(ntv_check_pd_20, 'ntv_check_pd_20')
ntv_justlda.save('ntv_ldamodel_20_topics')
save_obj(ntv_typical_groups, 'ntv_typical_groups')

In [21]:
ntv_groups_freq_dict = load_obj('ntv_groups_freq_dict')
ntv_final_data = load_obj('ntv_final_data')
ntv_subs_list = load_obj('ntv_subs_list')
ntv_group_doc_dict = load_obj('ntv_group_doc_dict')
ntv_group_clean_doc_dict = load_obj('ntv_group_clean_doc_dict')
ntv_justlda = LdaModel.load('ntv_ldamodel_20_topics')
ntv_check_pd_20 = load_obj('ntv_check_pd_20')
ntv_typical_groups = load_obj('ntv_typical_groups')

In [49]:
rain_final_data = get_final_data(rain)

0 profiles done
100 profiles done
200 profiles done
300 profiles done
400 profiles done
500 profiles done
600 profiles done
700 profiles done
800 profiles done
900 profiles done


In [12]:
temp_list = get_user_groups(rain_final_data, ntv)
rain_groups_freq_dict = temp_list[0]
rain_subs_list = temp_list[1]

NameError: name 'get_user_groups' is not defined

In [45]:
rain_group_doc_dict = get_group_doc_dict(rain_groups_freq_dict)

100 groups extracted
200 groups extracted
300 groups extracted
400 groups extracted
500 groups extracted
600 groups extracted
700 groups extracted
800 groups extracted
900 groups extracted
1000 groups extracted
1100 groups extracted
1200 groups extracted
1300 groups extracted
1400 groups extracted
1500 groups extracted
1600 groups extracted
1700 groups extracted
1800 groups extracted
1900 groups extracted
2000 groups extracted
2100 groups extracted
2200 groups extracted
2300 groups extracted
2400 groups extracted
2500 groups extracted
2600 groups extracted
2700 groups extracted
2800 groups extracted


In [48]:
rain_group_clean_doc_dict = get_clean_dict(rain_group_doc_dict)

100 docs processed
200 docs processed
300 docs processed
400 docs processed
500 docs processed
600 docs processed
700 docs processed
800 docs processed
900 docs processed
1000 docs processed
1100 docs processed
1200 docs processed
1300 docs processed
1400 docs processed
1500 docs processed
1600 docs processed
1700 docs processed
1800 docs processed
1900 docs processed
2000 docs processed
2100 docs processed
2200 docs processed
2300 docs processed
2400 docs processed
2500 docs processed
2600 docs processed
2700 docs processed
2800 docs processed


In [51]:
rain_justlda = get_lda(rain_group_clean_doc_dict)

LdaModel performance
0 человек жизнь любить жить знать друг сила глаз видеть рука
1 город гора светлый остров переодеться фото добрый луганский scent бузулук
2 ингредиент перец приготовление рецепт нарезать духовка перемешать салат куриный чеснок
3 человек друг знать найти ждать писать фото жизнь цена любой
4 инстраграмма 60х80 roach 30х40 коммунистический керамик 15х21 видеокассета редактироваться верещагин
5 найкращі техніка шановні інфраструктурить козацький депресії закарпаттить мерёжа засобіть пропозиції
6 новое человек ждать группа друг жизнь ноябрь история последний 2018
7 love make night 2018 time like good need onli noth
8 😍хотеть нoчный oптикa архыз cмoгнуть вcпoмнить ❤кака вамкак caмaть любитель
9 який післить дуже буде українить тілька такожа більша український человек
10 российский россия сотрудник тысяча страна проект государственный область москва организация
11 chevel пeред corvett convert road battlefield mark spider-man mujer franci
12 всeго овчинка 8900 дeлать дeнь н

In [54]:
rain_check_pd_20 = show_groups(rain_justlda, rain_group_clean_doc_dict)

print("Group distribution by the most relevant topic")
rain_group_index_percent = pd.Series.round(rain_check_pd_20.idxmax(axis=1).value_counts() * 1. / len(rain_check_pd_20), 2)
print(rain_group_index_percent)

Group distribution by the most relevant topic
0     0.31
6     0.16
3     0.15
10    0.11
16    0.10
2     0.02
9     0.02
13    0.02
7     0.02
19    0.01
14    0.01
18    0.01
1     0.01
8     0.01
11    0.01
12    0.01
5     0.01
15    0.01
17    0.00
4     0.00
dtype: float64


In [55]:
rain_typical_groups = show_most_response_group(rain_justlda, rain_group_clean_doc_dict, rain_check_pd_20)

The most typical groups for every topic
0 человек жизнь любить жить знать друг сила глаз видеть рука
Сверхспособности - реальные практики http://vk.com/club80235766
ЧИСТЫЙ РАЗУМ http://vk.com/club6629831
Свободная Жизнь | Эзотерика - Саморазвитие http://vk.com/club114675737
Путь воина. Ты можешь всё http://vk.com/club43702826
Тайные истории http://vk.com/club129613989
Другая Литература http://vk.com/club38744784
Психотерапия как искусство http://vk.com/club26768353
Эзотерика. Йога. Психология http://vk.com/club93958959
Сила Момента Сейчас (Экхарт Толле) http://vk.com/club4587548
Астролог Анна Куминская. Прогнозы, обучение. http://vk.com/club161250859

1 город гора светлый остров переодеться фото добрый луганский scent бузулук
Пиксельный рай http://vk.com/club169349452
Путешествия http://vk.com/club38363244
O B A F G K M http://vk.com/club145597354
Открытки http://vk.com/club28890647
Красивые места планеты: туризм http://vk.com/club111155620
sbw http://vk.com/club73930035
aquiver http:/

Новинки Музыки 2018 http://vk.com/club54500021

15 dima автогол футзал 2нарезать -3-4 влетать кряду 1222 касет 27-я
Кремень ТВ http://vk.com/club51451315
Что-то про мальчиков http://vk.com/club154485697
FASHION  мода http://vk.com/club33404310
УЛИЧНЫЕ ДРАКИ | UFC | БОКС | СПОРТ | GIF http://vk.com/club99628911
Дорожные Войны http://vk.com/club153344099
Дизайн &amp; Декор http://vk.com/club51696572
До | После http://vk.com/club115037408
BIG NATURAL TITS | БОЛЬШИЕ СИСЬКИ 18+ http://vk.com/club171180290
как выглядят люди http://vk.com/club101473197
Какой сегодня Щеп? http://vk.com/club170979714

16 ноябрь россия город человек фото улица группа пройти район найти
Самолет Девелопмент http://vk.com/club84850293
БашДТП | Уфа и Башкортостан http://vk.com/club48558367
ЭхоСтайл Профессиональный автозвук  Калининград http://vk.com/club119052896
BESTGOAL: ФУТБОЛ, ЛИГА ЧЕМПИОНОВ http://vk.com/club30156629
AlexAutoLive | Александров http://vk.com/club115968933
Полный Бак http://vk.com/club39236729
Л

In [62]:
save_obj(rain_group_doc_dict, 'rain_group_doc_dict')
save_obj(rain_groups_freq_dict, 'rain_groups_freq_dict')
save_obj(rain_final_data, 'rain_final_data')
save_obj(rain_subs_list, 'rain_subs_list')
save_obj(rain_group_clean_doc_dict, 'rain_group_clean_doc_dict')
rain_justlda.save('rain_ldamodel_20_topics')
save_obj(rain_check_pd_20, 'rain_check_pd_20')
save_obj(rain_typical_groups, 'rain_typical_groups')

In [63]:
rain_groups_freq_dict = load_obj('rain_groups_freq_dict')
rain_final_data = load_obj('rain_final_data')
rain_subs_list = load_obj('rain_subs_list')
rain_group_doc_dict = load_obj('rain_group_doc_dict')
rain_group_clean_doc_dict = load_obj('rain_group_clean_doc_dict')
rain_justlda = LdaModel.load('rain_ldamodel_20_topics')
rain_check_pd_20 = load_obj('rain_check_pd_20')
rain_typical_groups = load_obj('rain_typical_groups')

In [136]:
def range_pd20(justlda, group_clean_doc_dict, check_pd_20, group_index_percent):
    mycorp = ListTextCorpus(input=group_clean_doc_dict.values())
    
    position = {}
    index = 1

    for key, value in group_index_percent.items():
        position [key] = index
        index = index + 1

    terms_all = {}
    for i in range(20):
        terms = justlda.get_topic_terms(i)
        terms_all[i] = ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms))

    pd20_range = pd.DataFrame({'position':position, 'group_index_percent':group_index_percent, \
                                    'terms':terms_all})
    
    pd20_range = pd20_range.sort_values(by=['position'],ascending=True)
    pd20_range.set_index(['position'], inplace=True)
    
    return pd20_range

In [137]:
rain_pd20_range = range_pd20(rain_justlda, rain_group_clean_doc_dict, rain_check_pd_20, rain_group_index_percent)

In [138]:
ntv_pd20_range = range_pd20(ntv_justlda, ntv_group_clean_doc_dict, ntv_check_pd_20, ntv_group_index_percent)

In [144]:
ntv_vs_rain = ntv_pd20_range.join(rain_pd20_range, lsuffix='_ntv', rsuffix='_rain')
ntv_vs_rain.head()

,group_index_percent_ntv,terms_ntv,group_index_percent_rain,terms_rain
position,,,,
1,0.49,человек жизнь друг знать смотреть ждать любить...,0.31,человек жизнь любить жить знать друг сила глаз...
2,0.13,область район россия житель центр город сотруд...,0.16,новое человек ждать группа друг жизнь ноябрь и...
3,0.09,рубль цена писать телефон подарок скидка фото ...,0.15,человек друг знать найти ждать писать фото жиз...
4,0.07,вода масло простой способ приготовление тонкий...,0.11,российский россия сотрудник тысяча страна прое...
5,0.04,раунд досрочно весовой нурмагомедов макгрегор ...,0.10,ноябрь россия город человек фото улица группа ...


In [148]:
from plotly.offline import init_notebook_mode, iplot
import plotly
import plotly.graph_objs as go

init_notebook_mode(connected=True)


trace0 = go.Bar(
    x=ntv_vs_rain.index,
    y=ntv_vs_rain.group_index_percent_ntv,
    name='NTV',
    orientation = 'v'
)

trace1 = go.Bar(
    x=ntv_vs_rain.index,
    y=ntv_vs_rain.group_index_percent_rain,
    name='RAIN',
    orientation = 'v'
)

data = [trace0, trace1]
layout = {'title': 'Intrests'}

fig = go.Figure(data=data, layout=layout)

iplot(fig, show_link=False)